In [11]:
#import sqlalchemy
import sqlalchemy as sqa
from sqlalchemy.orm import declarative_base
from sqlalchemy import func
from sqlalchemy import ForeignKey
from sqlalchemy.orm import relationship

import timeit #for timing

from sqlalchemy.orm import sessionmaker

In [12]:
#creates engine to the database file
engine = sqa.create_engine("sqlite:///Data/movie.db")

In [13]:
#creates declarative base
Base = declarative_base()

In [14]:
#creates class for communicating with the movie table
class Movies(Base):
    __tablename__ = 'movies'
    
    id_movie = sqa.Column(sqa.Integer, primary_key=True)
    title_movie = sqa.Column(sqa.String)
    year_movie = sqa.Column(sqa.Integer)
    
    def __repr__(self):
        return "<Movie(id_movie='%i',title_movie='%s')>" % (self.id_movie, self.title_movie)

#creates class for communicating with the movie table
class Ratings(Base):
    __tablename__ = 'ratings'
    
    id_rating = sqa.Column(sqa.Integer, primary_key=True)
    id_movie = sqa.Column(sqa.Integer)
    id_user = sqa.Column(sqa.Integer)
    value_rating = sqa.Column(sqa.Float)
    timestamp_rating = sqa.Column(sqa.String)

    
    def __repr__(self):
        return "<Rating(id_rating='%i',id_movie='%i',id_user='%i',value_rating='%i',timestamp_rating='%s')>" % (self.id_rating,self.id_movie,self.id_user,self.value_rating, self.timestamp_rating)

#creates class for communicating with the movie table
class GenreList(Base):
    __tablename__ = 'genre_list'
    
    id_genre_item = sqa.Column(sqa.Integer, primary_key=True)
    id_movie = sqa.Column(sqa.Integer)
    name_genre = sqa.Column(sqa.String)
    
    
    def __repr__(self):
        return "<GenreList(id_genre_item='%i',id_movie='%i',name_genre='%s')>" % (self.id_genre_item,self.id_movie,self.id_user,self.name_genre)

    
#creates class temporary reference of average rating
class ReferenceRatings(Base):
    __tablename__ = 'reference_ratings'
    
    id_reference = sqa.Column(sqa.Integer, primary_key=True)
    average_rating = sqa.Column(sqa.Float)
    timestamp_update = sqa.Column(sqa.String)

    
    def __repr__(self):
        return "<Rating(id_movie='%i',average_rating='%i',timestamp_update='%s')>" % (self.id_movie,self.average_rating, self.timestamp_update)

#relationships

#Ratings.movie = relationship("Movies", back_populates="ratings")
#Movies.ratings = relationship("Ratings", order_by=Ratings.id_rating, back_populates="movie")

In [21]:
#connects to the database
Session = sessionmaker(bind=engine)
session = Session()

In [16]:
for m in session.query(Movies).order_by(Movies.id_movie)[1:30]:
    print(m.title_movie)

Jumanji
Grumpier Old Men
Waiting to Exhale
Father of the Bride Part II
Heat
Sabrina
Tom and Huck
Sudden Death
GoldenEye
American President, The
Dracula: Dead and Loving It
Balto
Nixon
Cutthroat Island
Casino
Sense and Sensibility
Four Rooms
Ace Ventura: When Nature Calls
Money Train
Get Shorty
Copycat
Assassins
Powder
Leaving Las Vegas
Othello
Now and Then
Persuasion
City of Lost Children, The (Cité des enfants perdus, La)
Shanghai Triad (Yao a yao yao dao waipo qiao)


In [17]:
start = timeit.default_timer()
for m in (session.query(Movies).filter(Movies.title_movie.like("%Jumanji%Welcome%"))[0:30]):
    print(m.title_movie)

stop = timeit.default_timer()

print('Time: ', stop - start) 

Jumanji: Welcome to the Jungle
Time:  0.009717299981275573


In [18]:
start = timeit.default_timer() #start timer

query = session.query(Movies.id_movie, Ratings.id_movie, func.avg(Ratings.value_rating)).filter(Movies.id_movie == Ratings.id_movie).order_by(Movies.id_movie).group_by(Movies.id_movie)[0:30]
print(query[:10])

stop = timeit.default_timer() #end timer

print('Time: ', stop - start) 

[(1, 1, 3.893707794587238), (2, 2, 3.2515271586594023), (3, 3, 3.142028126058963), (4, 4, 2.85354736424891), (5, 5, 3.0584343520573674), (6, 6, 3.854908898649748), (7, 7, 3.3636663369601054), (8, 8, 3.1145833333333335), (9, 9, 2.992050660199407), (10, 10, 3.42145763311516)]
Time:  31.270083500014152


In [9]:
start = timeit.default_timer() #start timer

query = session.query(Movies.id_movie, ReferenceRatings.average_rating,).filter(Movies.id_movie == ReferenceRatings.id_reference).order_by(Movies.id_movie)[0:30]
print(query[:10])

stop = timeit.default_timer() #end timer

print('Time: ', stop - start) 

[(1, 3.893707794587238), (2, 3.2515271586594023), (3, 3.142028126058963), (4, 2.85354736424891), (5, 3.0584343520573674), (6, 3.854908898649748), (7, 3.3636663369601054), (8, 3.1145833333333335), (9, 2.992050660199407), (10, 3.42145763311516)]
Time:  0.0017033999902196229


In [26]:
start = timeit.default_timer() #start timer
for m in session.query(Movies.title_movie, Movies.year_movie,ReferenceRatings.average_rating).filter(Movies.id_movie == ReferenceRatings.id_reference).order_by(Movies.id_movie)[0:30]:
    print(m[2])
stop = timeit.default_timer() #end timer

print('Time: ', stop - start) 

3.893707794587238
3.2515271586594023
3.142028126058963
2.85354736424891
3.0584343520573674
3.854908898649748
3.3636663369601054
3.1145833333333335
2.992050660199407
3.42145763311516
3.6571705198920315
2.6245656241646618
3.333527696793003
3.4234888364494465
2.7190217391304348
3.8237068028689416
3.948806325713417
3.3843505768515074
2.6421677802524126
2.869921875
3.5701620505454055
3.3200173216412256
3.145691333982473
3.18758389261745
3.677304434479322
3.606708513142409
3.4039315155358274
4.03
3.936724864539434
3.6367924528301887
Time:  0.0017934000061359257


In [36]:
start = timeit.default_timer()
for m in (session.query(GenreList.name_genre.distinct())):
    print(m[0].strip('\n'))

stop = timeit.default_timer()

print('Time: ', stop - start) 

Adventure
Animation
Children
Comedy
Fantasy
Romance
Drama
Comedy
Action
Crime
Thriller
Children
Action
Horror
Drama
Horror
Mystery
Sci-Fi
Fantasy
Sci-Fi
Romance
IMAX
Documentary
War
Musical
Mystery
Crime
Documentary
Musical
Thriller
Western
Film-Noir
War
Animation
Adventure
Film-Noir
Western
(no genres listed)
Time:  0.02149790001567453
